In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request():
    driver = webdriver.Chrome()
    url= 'https://www.bbc.com/news'
    driver.get(url)
    cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
    cookies_2= driver.find_element(By.CLASS_NAME, 'banner-button').click()    
    return driver

In [3]:
driver= url_request()

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
def keyword(word):
    search_bar  = driver.find_element(By.LINK_TEXT, 'Search BBC').click()
    search_bar  = driver.find_element(By.ID, 'search-input')
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)
    
    link_lst=[]
    year_control=[]

    while len(link_lst)<3:
        lnks           = driver.find_elements(By.TAG_NAME,"a")
        lst_bbc        = [lnk.get_attribute('href') for lnk in lnks]
        lst_bbc_       = [url for url in lst_bbc if '/news/' in url and '/help' not in url and '/live/' not in url]
        search_bar = driver.find_element(By.LINK_TEXT, 'next page').click()
        for i in lst_bbc_:
            link_lst.append(i)

        link_lst= list(set(link_lst))
        
        list_soup=[]
        for url in link_lst:
            html = requests.get(url)
            soup = BeautifulSoup(html.content, "html.parser")
            article=soup.getText().replace('\'', "´").strip()
            #string_to_find = str(soup.find('div', {'class': 'Divider'}))
            #article = article[article.find(string_to_find):]
            
            article = article[:article.find('More on this story')]
            list_soup.append(article)
            
            

        
        dict_bbc={'article': list_soup,
                 'link':link_lst}
    return dict_bbc


In [ ]:
dict_=keyword('Russia')

In [ ]:
df= pd.DataFrame(dict_)

In [ ]:
df['article'][1]

        
        day_list=[]
        month_list=[]
        year_list=[]
        year_control=[]

        for soup in list_soup:
            date_str = soup.find('span', {'class': 'date'}).text.strip()
            day, month, year, time = date_str.split()
            day = day.zfill(2)
            month = month.zfill(2).replace(',','')
            year= year
            
            day_list.append(day)
            month_list.append(month)
            year_list.append(year)
            year_control=[i for i in year_list if i == '2020']
                    
    dict_={
        'link': lst_link,
        'article': art_list,
        'year' : year_list,
        'month': month_list,
        'day': day_list
    }
    df= pd.DataFrame(dict_)